In [5]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [222]:
# Load dataset and check proportion of class imbalance
train = pd.read_csv(r'C:\Users\hp\Downloads\Train Dataset .csv')
test =  pd.read_csv(r'C:\Users\hp\Downloads\Test Dataset.csv')

## Feature Engineering

In [223]:
train['high_chol_if_exang'] = ((train['chol'] > 240) & (train['exang'] == 1)).astype(int)
train['high_chol_ifnot_exang'] = ((train['chol'] > 240) & (train['exang'] == 0)).astype(int)
train['high_trestbps_if_exang'] = ((train['trestbps'] > 130) & (train['exang'] == 1)).astype(int)
train['high_trestbps_ifnot_exang'] = ((train['trestbps'] > 130) & (train['exang'] == 0)).astype(int)
train['low_trestbps_if_exang'] = ((train['trestbps'] < 130) & (train['exang'] == 1)).astype(int)
train['low_trestbps_ifnot_exang'] = ((train['trestbps'] < 130) & (train['exang'] == 0)).astype(int)
train['ttal'] = train['high_chol_if_exang'] + train['high_trestbps_if_exang'] 
train['exang_thalach_interaction'] = train['exang'] * train['thalach']
train['exang_age_interaction'] = train['exang'] * train['Age']
train['high_age_if_exang'] = ((train['chol'] >  60) & (train['exang'] == 1)).astype(int)
train['high_age_ifnot_exang'] = ((train['chol'] >  60) & (train['exang'] == 0)).astype(int)
train['male_if_exang'] = ((train['Sex'] == 1) & (train['exang'] == 1)).astype(int)
train['fmale_if_exang'] = ((train['Sex'] == 0) & (train['exang'] == 1)).astype(int)
train['male_ifnot_exang'] = ((train['Sex'] == 1) & (train['exang'] == 0)).astype(int)
train['fmale_ifnot_exang'] = ((train['Sex'] == 0) & (train['exang'] == 0)).astype(int)

In [224]:
test.rename(columns = {'age':'Age', 'sex':'Sex', 'id':'Id'}, inplace=True)

In [225]:
test['high_chol_if_exang'] = ((test['chol'] > 240) & (test['exang'] == 1)).astype(int)
test['high_chol_ifnot_exang'] = ((test['chol'] > 240) & (test['exang'] == 0)).astype(int)
test['high_trestbps_if_exang'] = ((test['trestbps'] > 130) & (test['exang'] == 1)).astype(int)
test['high_trestbps_ifnot_exang'] = ((test['trestbps'] > 130) & (test['exang'] == 0)).astype(int)
test['low_trestbps_if_exang'] = ((test['trestbps'] < 130) & (test['exang'] == 1)).astype(int)
test['low_trestbps_ifnot_exang'] = ((test['trestbps'] < 130) & (test['exang'] == 0)).astype(int)
test['ttal'] = test['high_chol_if_exang'] + test['high_trestbps_if_exang'] 
test['exang_thalach_interaction'] = test['exang'] * test['thalach']
test['exang_age_interaction'] = test['exang'] * test['Age']
test['high_age_if_exang'] = ((test['chol'] >  60) & (test['exang'] == 1)).astype(int)
test['high_age_ifnot_exang'] = ((test['chol'] >  60) & (test['exang'] == 0)).astype(int)
test['male_if_exang'] = ((test['Sex'] == 1) & (test['exang'] == 1)).astype(int)
test['fmale_if_exang'] = ((test['Sex'] == 0) & (test['exang'] == 1)).astype(int)
test['male_ifnot_exang'] = ((test['Sex'] == 1) & (test['exang'] == 0)).astype(int)
test['fmale_ifnot_exang'] = ((test['Sex'] == 0) & (test['exang'] == 0)).astype(int)

### Feature Selection
Using Mutual_Info Classif

In [226]:
import pandas as pd
from sklearn.feature_selection import mutual_info_classif

X = train.drop(columns=['target'])
y = train['target']
mutual_info = mutual_info_classif(X, y, random_state=1)
feature_scores = pd.Series(mutual_info, index=X.columns).sort_values(ascending=False)
print(feature_scores)

train_set = feature_scores.index[:15].tolist()

high_age_ifnot_exang         0.155391
high_age_if_exang            0.152533
exang                        0.152417
exang_age_interaction        0.152369
exang_thalach_interaction    0.146698
ttal                         0.128685
high_chol_if_exang           0.102403
high_trestbps_if_exang       0.085970
high_chol_ifnot_exang        0.065211
male_if_exang                0.064455
cp                           0.061066
fmale_if_exang               0.058343
high_trestbps_ifnot_exang    0.048956
fmale_ifnot_exang            0.046091
low_trestbps_if_exang        0.040454
male_ifnot_exang             0.031989
low_trestbps_ifnot_exang     0.017719
thalach                      0.003472
chol                         0.003353
Age                          0.001172
thal                         0.000952
ca                           0.000504
restecg                      0.000183
slope                        0.000000
oldpeak                      0.000000
fbs                          0.000000
trestbps    

### Train Base Model

In [227]:
# Base Model using Logistics Regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import KFold, cross_val_score
import numpy as np

X = train.drop(columns=['target'])  # separate into dependent and independent(x) variables
y = train['target']
X = X[train_set]
#test = test[train_set]

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #split dataset

In [228]:
def base_model(x, y, x_test = x_test, y_test = y_test):
    model = LogisticRegression(random_state=42, max_iter=30)
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)


    print(f'test_acc: {accuracy_score(y_test, y_pred)}')
base_model(x_train, y_train)

test_acc: 0.8254620123203286


## Train XGBoost Model

In [230]:
from hyperopt import fmin, tpe, hp, STATUS_OK
from sklearn.model_selection import cross_val_score, StratifiedKFold
import xgboost as xgb
import numpy as np

space = {
    'n_estimators': hp.choice('n_estimators', [100, 200, 300, 400]),  
    'max_depth': hp.quniform('max_depth', 3, 10, 1),  # this needs to be cast to an int                   
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),           
    'subsample': hp.uniform('subsample', 0.7, 1.0),                    
    'gamma': hp.uniform('gamma', 0, 0.5),                              
    'reg_alpha': hp.uniform('reg_alpha', 0, 1.0),                      
    'reg_lambda': hp.uniform('reg_lambda', 0.1, 1.0)                   
}

def objective(params):
    params['max_depth'] = int(params['max_depth'])
    model = xgb.XGBClassifier(**params, random_state=42, eval_metric='auc', objective='binary:logistic')
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=skf, scoring='accuracy')
    accuracy = np.mean(scores)
    loss = 1 - accuracy
    return {'loss': loss, 'status': STATUS_OK}
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100)

print("Best hyperparameters:", best)


100%|██████████| 100/100 [03:42<00:00,  2.22s/trial, best loss: 0.18376074659784836]
Best hyperparameters: {'gamma': 0.3809586511326805, 'learning_rate': 0.010231006731096733, 'max_depth': 10.0, 'n_estimators': 2, 'reg_alpha': 0.6370836893631369, 'reg_lambda': 0.15105866637207888, 'subsample': 0.9307334100822032}


In [231]:
from sklearn.model_selection import StratifiedKFold, cross_val_score
skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=42)
model = xgb.XGBClassifier(gamma = 0.3809586511326805, learning_rate = 0.010231006731096733, max_depth = 10, n_estimators = 2, reg_alpha =  0.6370836893631369, 
                          reg_lambda = 0.15105866637207888, subsample = 0.9307334100822032)
cross_val_score(model, X, y, cv = skf, scoring= 'accuracy').mean()

0.8135013278518807